# How many max features?

In [12]:
!pip install mlflow boto3 awscli

  Using cached docutils-0.19-py3-none-any.whl.metadata (2.7 kB)
Using cached docutils-0.19-py3-none-any.whl (570 kB)
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.2.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 which is incompatible.


In [13]:
pip install "docutils>=0.20,<0.22" --upgrade

  Using cached docutils-0.21.2-py3-none-any.whl.metadata (2.8 kB)
Using cached docutils-0.21.2-py3-none-any.whl (587 kB)
  Attempting uninstall: docutils
    Found existing installation: docutils 0.19
    Uninstalling docutils-0.19:
      Successfully uninstalled docutils-0.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.44.42 requires docutils<=0.19,>=0.18.1, but you have docutils 0.21.2 which is incompatible.


In [14]:
pip check

ipython 7.34.0 requires jedi, which is not installed.
awscli 1.44.42 has requirement docutils<=0.19,>=0.18.1, but you have docutils 0.21.2.


In [15]:
!aws configure

AWS Access Key ID [****************GDOY]: 
AWS Secret Access Key [****************sU/U]: 
Default region name [ap-south-1]: 
Default output format [None]: 


In [20]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-13-204-252-144.ap-south-1.compute.amazonaws.com:5000")

In [21]:
import requests
print(requests.get("http://ec2-13-204-252-144.ap-south-1.compute.amazonaws.com:5000").status_code)

200


In [22]:
# Set or create an experiment
mlflow.set_experiment("Exp 3 - TfIdf Trigram max_features")

2026/02/19 10:46:09 INFO mlflow.tracking.fluent: Experiment with name 'Exp 3 - TfIdf Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlfow-bucket-26/3', creation_time=1771497969385, experiment_id='3', last_update_time=1771497969385, lifecycle_stage='active', name='Exp 3 - TfIdf Trigram max_features', tags={}>

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
df = pd.read_csv('/content/drive/MyDrive/reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [27]:
# Step 1: Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)  # Trigram setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2026/02/19 10:48:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run TFIDF_Trigrams_max_features_1000 at: http://ec2-13-204-252-144.ap-south-1.compute.amazonaws.com:5000/#/experiments/3/runs/68b5d42df18e4a7d9586c8ed7f5c38c3
🧪 View experiment at: http://ec2-13-204-252-144.ap-south-1.compute.amazonaws.com:5000/#/experiments/3


2026/02/19 10:49:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run TFIDF_Trigrams_max_features_2000 at: http://ec2-13-204-252-144.ap-south-1.compute.amazonaws.com:5000/#/experiments/3/runs/eb36eefcb262431693f8bebc7672471f
🧪 View experiment at: http://ec2-13-204-252-144.ap-south-1.compute.amazonaws.com:5000/#/experiments/3


2026/02/19 10:49:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


...

🏃 View run TFIDF_Trigrams_max_features_8000 at: http://ec2-13-204-252-144.ap-south-1.compute.amazonaws.com:5000/#/experiments/3/runs/d3fcc5b87b6842e792c062ac6b11f03a
🧪 View experiment at: http://ec2-13-204-252-144.ap-south-1.compute.amazonaws.com:5000/#/experiments/3


2026/02/19 10:54:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run TFIDF_Trigrams_max_features_9000 at: http://ec2-13-204-252-144.ap-south-1.compute.amazonaws.com:5000/#/experiments/3/runs/f9c75657c5d94e96a4cb4acc88271f4d
🧪 View experiment at: http://ec2-13-204-252-144.ap-south-1.compute.amazonaws.com:5000/#/experiments/3


2026/02/19 10:54:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run TFIDF_Trigrams_max_features_10000 at: http://ec2-13-204-252-144.ap-south-1.compute.amazonaws.com:5000/#/experiments/3/runs/f788b0883d2c4916a29765845431d79f
🧪 View experiment at: http://ec2-13-204-252-144.ap-south-1.compute.amazonaws.com:5000/#/experiments/3
